In [2]:
from PIL import Image, ImageFilter, ImageDraw
import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning)

In [3]:
def raster_blur_effect(input_path, output_path, slice_count=15, blur_radius=10, highlight_intensity=0.1, shadow_intensity=0.1):
    """
    Convert the input wallpaper into a rasterized frosted glass effect with added highlights and shadows.

    Parameters:
        input_path (str): File path of the input wallpaper.
        output_path (str): File path for saving the output wallpaper.
        slice_count (int): Number of vertical slices.
        blur_radius (int): Radius of the Gaussian blur. Default is 10.
        highlight_intensity (float): Intensity of the highlight effect. Default is 0.1.
        shadow_intensity (float): Intensity of the shadow effect. Default is 0.1.
    """
    # Open the input wallpaper
    image = Image.open(input_path)
    width, height = image.size  # Get the width and height of the wallpaper

    # Fixed overlap ratio of 0.5
    overlap_ratio = 0.5

    # Calculate the width of each original slice and the step width
    num_slices = (slice_count + 1) // 2
    original_slice_width = width / num_slices
    step_width = width / (num_slices / overlap_ratio)

    # Round the widths to integers
    original_slice_width = int(original_slice_width)
    step_width = int(step_width)

    # Calculate the compressed width of each slice
    compressed_slice_width = int(width / slice_count)

    # Store the slices in a list
    slices = []

    for i in range(slice_count):
        # Calculate the starting and ending positions of each slice
        start_x = int(i * step_width)
        end_x = start_x + original_slice_width

        # Adjust the ending position if it exceeds the image width
        if end_x > width:
            end_x = width

        # Crop the slice from the image
        slice_strip = image.crop((start_x, 0, end_x, height))

        # Apply Gaussian blur if blur_radius > 0
        if blur_radius > 0:
            slice_strip = slice_strip.filter(ImageFilter.GaussianBlur(blur_radius))

        # Add highlight effect
        if highlight_intensity > 0:
            draw = ImageDraw.Draw(slice_strip, "RGBA")
            highlight_width = int(original_slice_width * 0.2)  # Highlight width is 20% of the slice width
            gradient = Image.new('RGBA', (highlight_width, height), (255, 255, 255, 0))
            gradient_draw = ImageDraw.Draw(gradient)

            for x in range(highlight_width):
                alpha = int(255 * highlight_intensity * (1 - x / highlight_width))
                gradient_draw.line([(x, 0), (x, height)], fill=(255, 255, 255, alpha))

            slice_strip.paste(gradient, (0, 0), gradient)

        # Add shadow effect
        if shadow_intensity > 0:
            draw = ImageDraw.Draw(slice_strip, "RGBA")
            shadow_width = int(original_slice_width * 0.1)  # Shadow width is 10% of the slice width
            gradient = Image.new('RGBA', (shadow_width, height), (0, 0, 0, 0))
            gradient_draw = ImageDraw.Draw(gradient)

            for x in range(shadow_width):
                alpha = int(255 * shadow_intensity * (x / shadow_width))
                gradient_draw.line([(x, 0), (x, height)], fill=(0, 0, 0, alpha))

            slice_strip.paste(gradient, (original_slice_width - shadow_width, 0), gradient)

        # Compress the slice to the target width
        compressed_strip = slice_strip.resize((compressed_slice_width, height), Image.LANCZOS)

        slices.append(compressed_strip)

    # Create a blank canvas without specifying size initially
    final_image_width = compressed_slice_width * slice_count
    final_image = Image.new('RGB', (final_image_width, height))

    # Paste each slice onto the canvas
    current_x = 0
    for slice_strip in slices:
        final_image.paste(slice_strip, (current_x, 0))
        current_x += compressed_slice_width

    # Resize the final image to match the original dimensions
    final_image = final_image.resize((width, height), Image.LANCZOS)

    # Save the final output image
    final_image.save(output_path)

In [29]:
raster_blur_effect(
    input_path='photos/65353.jpg',
    output_path='photos/65353_rasterization.jpg'
)

In [30]:
raster_blur_effect(
    input_path='photos/65368.jpg',
    output_path='photos/65368_rasterization.jpg',
    slice_count=23,
    blur_radius=15,
    highlight_intensity=0.1
)

In [38]:
raster_blur_effect(
    input_path='photos/smartisanR1.jpg',
    output_path='photos/smartisanR1_rasterization.jpg',
    slice_count=21,
    blur_radius=20,
    highlight_intensity=0.1,
    shadow_intensity=0.05
)

In [39]:
raster_blur_effect(
    input_path='photos/flower1.jpg',
    output_path='photos/flower1_rasterization.jpg',
    slice_count=15,
    blur_radius=10,
    highlight_intensity=0.1,
    shadow_intensity=0.1
)

In [41]:
raster_blur_effect(
    input_path='photos/flower2.jpg',
    output_path='photos/flower2_rasterization.jpg',
    slice_count=21,
    blur_radius=10,
    highlight_intensity=0.1,
    shadow_intensity=0.1
)